# Data exploration of Netflix move dataset
https://www.kaggle.com/datasets/netflix-inc/netflix-prize-data

## Import libraries

In [17]:
from dotenv import load_dotenv
from io import BytesIO
import pandas as pd
import os

In [18]:
## change working directory
os.chdir('/app')
## show current working directory
os.getcwd()

'/app'

In [ ]:
# continue loading package functions
from ml.utils.minio_utils import get_or_create_bucket, list_objects_in_bucket


In [20]:
# load env variables
load_dotenv()

True

## Load sample data
Read example file to see how it looks like

In [ ]:
# list all objects in bucket
bucket = get_or_create_bucket(os.getenv('MINIO_BUCKET_NAME', ""))
for file_path in list_objects_in_bucket(os.getenv('MINIO_BUCKET_NAME', "")):
    print(file_path)

data/raw/README
data/raw/combined_data_1.txt
data/raw/combined_data_2.txt
data/raw/combined_data_3.txt
data/raw/combined_data_4.txt
data/raw/movie_titles.csv
data/raw/probe.txt
data/raw/qualifying.txt


In [ ]:
# read and print 10 rows of data file
obj_raw = bucket.Object("data/bronze/combined_data_1.txt")
body_data = obj_raw.get()["Body"]

lines = []
for _ in range(10):
    line = body_data.readline().decode("utf-8")
    if not line:
        break
    lines.append(line)

print("".join(lines))


1:
1488844,3,2005-09-06
822109,5,2005-05-13
885013,4,2005-10-19
30878,4,2005-12-26
823519,3,2004-05-03
893988,3,2005-11-17
124105,4,2004-08-05
1248029,3,2004-04-22
1842128,4,2004-05-09



In [ ]:
# read and print 10 rows of dict
obj_raw = bucket.Object("data/bronze/movie_titles.csv")
body_dict = obj_raw.get()["Body"]

lines = []
for _ in range(10):
    line = body_dict.readline().decode("utf-8")
    if not line:
        break
    lines.append(line)

print("".join(lines))


1,2003,Dinosaur Planet
2,2004,Isle of Man TT 2004 Review
3,1997,Character
4,1994,Paula Abdul's Get Up & Dance
5,2004,The Rise and Fall of ECW
6,1997,Sick
7,1992,8 Man
8,2004,What the #$*! Do We Know!?
9,1991,Class of Nuke 'Em High 2
10,2001,Fighter



## 2. Load data to pandas dataframe
For further analysis and manipulations

In [ ]:
obj_raw = bucket.Object("data/bronze/combined_data_1.txt")
body_data = obj_raw.get()["Body"]

df_iter = pd.read_csv(
    body_data,
    chunksize=50000,
    header=None,
    sep="|"
)
for chunk in df_iter:
    print(chunk.head())
    break

                      0
0                    1:
1  1488844,3,2005-09-06
2   822109,5,2005-05-13
3   885013,4,2005-10-19
4    30878,4,2005-12-26


In [25]:
len(chunk)

50000

## 3. Preprocess data
Final preprocessed data format should be (Movie_id, User_id, rating, Date)

In [26]:
# adds column for indication rows with movie_id
chunk["movie_id_ind"] = chunk[0].str.contains(":")
chunk.head()

,0,movie_id_ind
0,1:,True
1,"1488844,3,2005-09-06",False
2,"822109,5,2005-05-13",False
3,"885013,4,2005-10-19",False
4,"30878,4,2005-12-26",False


In [27]:
# running total of movie ind
chunk["movie_cum_sum"] = chunk["movie_id_ind"].cumsum()
chunk.head()

,0,movie_id_ind,movie_cum_sum
0,1:,True,1
1,"1488844,3,2005-09-06",False,1
2,"822109,5,2005-05-13",False,1
3,"885013,4,2005-10-19",False,1
4,"30878,4,2005-12-26",False,1


In [28]:
# create df for only movies
movie_ids = chunk[chunk["movie_id_ind"]][[0, "movie_cum_sum"]]
movie_ids[0] = movie_ids[0].str.replace(":", "")
movie_ids.columns = ["movie_id", "movie_cum_sum"]
# movie_ids.set_index(0, inplace=True)
movie_ids.head()

,movie_id,movie_cum_sum
0,1,1
548,2,2
694,3,3
2707,4,4
2850,5,5


In [29]:
chunk.head()

,0,movie_id_ind,movie_cum_sum
0,1:,True,1
1,"1488844,3,2005-09-06",False,1
2,"822109,5,2005-05-13",False,1
3,"885013,4,2005-10-19",False,1
4,"30878,4,2005-12-26",False,1


In [30]:
# filter out movie_ids rows from df
chunk = chunk[~chunk["movie_id_ind"]]
chunk.head()

,0,movie_id_ind,movie_cum_sum
1,"1488844,3,2005-09-06",False,1
2,"822109,5,2005-05-13",False,1
3,"885013,4,2005-10-19",False,1
4,"30878,4,2005-12-26",False,1
5,"823519,3,2004-05-03",False,1


In [31]:
# split rows data into user_id, rating, date
chunk[["User_id", "Rating", "Date"]] = chunk.loc[:, 0].str.split(pat=",", expand=True)
chunk.head()

,0,movie_id_ind,movie_cum_sum,User_id,Rating,Date
1,"1488844,3,2005-09-06",False,1,1488844,3,2005-09-06
2,"822109,5,2005-05-13",False,1,822109,5,2005-05-13
3,"885013,4,2005-10-19",False,1,885013,4,2005-10-19
4,"30878,4,2005-12-26",False,1,30878,4,2005-12-26
5,"823519,3,2004-05-03",False,1,823519,3,2004-05-03


In [33]:
# combine movie_ids and chunk
final_df = chunk.merge(movie_ids, how="left", on="movie_cum_sum")
final_df.drop(columns=[0, "movie_id_ind", "movie_cum_sum"], inplace=True)
final_df.head()

,User_id,Rating,Date,movie_id
0,1488844,3,2005-09-06,1
1,822109,5,2005-05-13,1
2,885013,4,2005-10-19,1
3,30878,4,2005-12-26,1
4,823519,3,2004-05-03,1
